In [1]:
import numpy as np
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_json('/Users/laurenfinkelstein/Metis/Projects/Project 3/credit_payment_default/results-20190430-210539.json')

# Data cleaning

In [ ]:
df.head()


In [ ]:
df.info()


In [ ]:
# is there only one entry per unique user id?
# answer = no duplicates

new = df.drop_duplicates()
new.count()
# df_filtered.groupby('id').count()
# df_filtered['id'].value_counts() > 1

In [3]:
# remove unecessary columns and reorder necessary columns

new_columns = ['default_payment_next_month', 'age', 'sex', 'marital_status', 'education_level',  
               'bill_amt_1', 'bill_amt_2', 'bill_amt_3', 'bill_amt_4', 'bill_amt_5', 'bill_amt_6', 
               'pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6', 
               'pay_amt_1', 'pay_amt_2', 'pay_amt_3', 'pay_amt_4', 'pay_amt_5', 'pay_amt_6', 
               'limit_balance']

model_data = df[new_columns]


In [ ]:
model_data.head()


In [ ]:
# figuring out what the different variables mean
# pay_0 etc. have additional labels other than labels described in GCP (e.g., 0, -1), but leave as is?

temp_columns = ['default_payment_next_month', 'age', 'sex', 'marital_status', 'education_level',  
               'bill_amt_1',
               'pay_0', 
               'pay_amt_1',
               'limit_balance']

df_temp = model_data[temp_columns]
df_temp.head()
# df_temp.pay_0.value_counts()

In [4]:
# where pay_columns have a value < 0, change to 0
# ASSUMPTION: if pay columns have a negative value, customer paid on time

pay_columns = ['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']

for column in pay_columns:
    model_data.loc[model_data[column] < 0, column] = 0
    
    

/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [ ]:
model_data.head()


In [5]:
# sex: replace female=2 with female=0 to prepare for creating dummy variables

# model_data['sex'] = model_data['sex'].map({1: 'male', 2: 'female'})
# model_data.sex

model_data['sex2'] = np.where(model_data['sex'] > 1, 0, model_data['sex']) # female = 0, male = 1

# def gender_convert(num):
#     if num == 1:
#         return 'male'
#     if num == 2:
#         return 'female'
# model_data['sex'].map(gender_convert)



/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
model_data['sex2'].value_counts()


In [ ]:
model_data.columns

In [6]:
model_data.drop(columns=['sex'], axis=1, inplace=True)
model_data.columns



/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Index(['default_payment_next_month', 'age', 'marital_status',
       'education_level', 'bill_amt_1', 'bill_amt_2', 'bill_amt_3',
       'bill_amt_4', 'bill_amt_5', 'bill_amt_6', 'pay_0', 'pay_2', 'pay_3',
       'pay_4', 'pay_5', 'pay_6', 'pay_amt_1', 'pay_amt_2', 'pay_amt_3',
       'pay_amt_4', 'pay_amt_5', 'pay_amt_6', 'limit_balance', 'sex2'],
      dtype='object')

In [7]:
model_data.rename(columns={'sex2': 'sex'}, inplace=True)
model_data.head()



/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,default_payment_next_month,age,marital_status,education_level,bill_amt_1,bill_amt_2,bill_amt_3,bill_amt_4,bill_amt_5,bill_amt_6,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,limit_balance,sex
0,0,39,2,1,47174,47974,48630,50803,30789,15874,0,0,0,0,0,0,1800,2000,3000,2000,2000,2000,50000,1
1,0,29,2,1,48088,45980,44231,32489,26354,20221,0,0,0,0,0,0,2000,2010,3000,3000,3000,1000,110000,0
2,0,36,2,1,78630,68921,46512,40335,37165,22156,0,0,0,2,0,0,10076,4018,14,2051,2000,0,270000,1
3,0,45,1,1,58180,59134,61156,62377,63832,65099,0,0,0,0,0,0,2886,2908,2129,2354,2366,2291,130000,1
4,0,24,2,1,42058,35340,22110,19837,19855,20151,0,0,0,0,0,0,1367,1606,692,709,721,692,50000,1


In [8]:
# replace marital_status number representations with categorical labels to prepare for dummy encoding

model_data['marital_status'] = model_data['marital_status'].map({1: 'married', 2: 'single', 3: 'others'})
model_data.marital_status


/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0        single
1        single
2        single
3       married
4        single
5        single
6        single
7       married
8       married
9        single
10       single
11       single
12       single
13       single
14      married
15      married
16       single
17       single
18       single
19      married
20       single
21       single
22       single
23      married
24       single
25       single
26       single
27      married
28       single
29       single
30       single
31       single
32       single
33      married
34       single
35       single
36       single
37       single
38      married
39      married
40       single
41       single
42       single
43       single
44      married
45       single
46       single
47      married
48      married
49       single
50      married
51      married
52      married
53       single
54       single
55       single
56       single
57       single
58       single
59      married
60      married
61      married
62      

In [9]:
# replace education level numerical label with string to prepare for dummy variables
# Education Level (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)

model_data['education_level'] = model_data['education_level'].map({1: 3, 2: 2, 3: 1, 4: 0, 5: 0, 6: 0})
model_data.education_level


/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


0       3.0
1       3.0
2       3.0
3       3.0
4       3.0
5       3.0
6       3.0
7       3.0
8       3.0
9       3.0
10      2.0
11      2.0
12      2.0
13      2.0
14      2.0
15      2.0
16      2.0
17      2.0
18      2.0
19      2.0
20      2.0
21      2.0
22      1.0
23      1.0
24      1.0
25      3.0
26      1.0
27      3.0
28      3.0
29      3.0
30      3.0
31      3.0
32      2.0
33      2.0
34      2.0
35      2.0
36      2.0
37      2.0
38      2.0
39      2.0
40      2.0
41      2.0
42      2.0
43      2.0
44      1.0
45      1.0
46      1.0
47      1.0
48      1.0
49      1.0
50      1.0
51      2.0
52      3.0
53      3.0
54      3.0
55      3.0
56      3.0
57      3.0
58      3.0
59      3.0
60      3.0
61      2.0
62      2.0
63      2.0
64      2.0
65      2.0
66      2.0
67      2.0
68      2.0
69      2.0
70      2.0
71      2.0
72      2.0
73      2.0
74      2.0
75      1.0
76      1.0
77      1.0
78      3.0
79      3.0
80      3.0
81      3.0
82      3.0
83  

In [10]:
model_data.head()


,default_payment_next_month,age,marital_status,education_level,bill_amt_1,bill_amt_2,bill_amt_3,bill_amt_4,bill_amt_5,bill_amt_6,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,limit_balance,sex
0,0,39,single,3.0,47174,47974,48630,50803,30789,15874,0,0,0,0,0,0,1800,2000,3000,2000,2000,2000,50000,1
1,0,29,single,3.0,48088,45980,44231,32489,26354,20221,0,0,0,0,0,0,2000,2010,3000,3000,3000,1000,110000,0
2,0,36,single,3.0,78630,68921,46512,40335,37165,22156,0,0,0,2,0,0,10076,4018,14,2051,2000,0,270000,1
3,0,45,married,3.0,58180,59134,61156,62377,63832,65099,0,0,0,0,0,0,2886,2908,2129,2354,2366,2291,130000,1
4,0,24,single,3.0,42058,35340,22110,19837,19855,20151,0,0,0,0,0,0,1367,1606,692,709,721,692,50000,1


In [18]:
dummy = pd.get_dummies(model_data['marital_status'], drop_first = True)
model_data = pd.concat([model_data,dummy], axis=1).drop(columns='marital_status')
model_data.head()


,default_payment_next_month,age,education_level,bill_amt_1,bill_amt_2,bill_amt_3,bill_amt_4,bill_amt_5,bill_amt_6,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,limit_balance,sex,others,single
0,0,39,3.0,47174,47974,48630,50803,30789,15874,0,0,0,0,0,0,1800,2000,3000,2000,2000,2000,50000,1,0,1
1,0,29,3.0,48088,45980,44231,32489,26354,20221,0,0,0,0,0,0,2000,2010,3000,3000,3000,1000,110000,0,0,1
2,0,36,3.0,78630,68921,46512,40335,37165,22156,0,0,0,2,0,0,10076,4018,14,2051,2000,0,270000,1,0,1
3,0,45,3.0,58180,59134,61156,62377,63832,65099,0,0,0,0,0,0,2886,2908,2129,2354,2366,2291,130000,1,0,0
4,0,24,3.0,42058,35340,22110,19837,19855,20151,0,0,0,0,0,0,1367,1606,692,709,721,692,50000,1,0,1


In [19]:
model_data.dropna(axis=0, inplace = True)



In [20]:
model_data.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 2963 entries, 0 to 2964
Data columns (total 25 columns):
default_payment_next_month    2963 non-null int64
age                           2963 non-null int64
education_level               2963 non-null float64
bill_amt_1                    2963 non-null int64
bill_amt_2                    2963 non-null int64
bill_amt_3                    2963 non-null int64
bill_amt_4                    2963 non-null int64
bill_amt_5                    2963 non-null int64
bill_amt_6                    2963 non-null int64
pay_0                         2963 non-null int64
pay_2                         2963 non-null int64
pay_3                         2963 non-null int64
pay_4                         2963 non-null int64
pay_5                         2963 non-null int64
pay_6                         2963 non-null int64
pay_amt_1                     2963 non-null int64
pay_amt_2                     2963 non-null int64
pay_amt_3                     2963 non-nu

In [21]:
with open ('cleaned_cc_default_data', 'wb') as f:
    pickle.dump(model_data, f)
    
    